# **Data and Evidence CoP**
## Introduction to APIs

We will start by bringing in packages to our coding environment. These packages contain functions that allow us to do different things within our code.

We will be using:
- `requests`: a package for interacting with APIs
- `pandas`: given the short-form `pd`, is a package for working with dataframes, a type of data structure in Python that resembles a table

In [ ]:
import requests
import pandas as pd

Now that we have installed all the functions that we need, we can start to define some variables that we need to access our data.

In [ ]:
base_url = "http://environment.data.gov.uk/hydrology/id/stations"
print(base_url)

If we visit that link, we can see it returns a lot of stations, 100 in fact. This is because that is the soft limit of the API, meaning that unless we specify a limit, or there are fewer than 1,000 items in the request, it will display the first 100 entries.
Let's set a limit so we only get the first ten entries.

Note that we have already defined the base url and the stations url, so we don't need to do this again.
To add a query to our URL, we first need to add a `?` and then we define our query. We put an `&` between each query if we have more than one.

In [ ]:
limit_query = "?_limit=10"
stations_10_url = base_url + limit_query
print(stations_10_url)

Now that we're familiar with constructing a URL for accessing data, let's expand on this work by using the `requests` library to bring this data into our environment.
We use the `requests.get()` function to do this. 
This link provides more information on this method: 
https://www.w3schools.com/PYTHON/ref_requests_get.asp#:~:text=The%20get%28%29%20method%20sends%20a%20GET%20request%20to,timeout%3D2.50%29%20Required.%20The%20url%20of%20the%20request%20Optional
In summary, the get() method sends a GET request. We provide it with a URL, and it accesses this URL, and simply "gets" some information from it.
Let's put our stations_10_url into the function. 

In [ ]:
stations_10_response = requests.get(stations_10_url)
print(stations_10_response)

If we print our return from this function, it tells us that the status code of the return was 200. This means the request was successful.
Let's view the response.

In [ ]:
if stations_10_response.status_code == 200:
    stations_10_json = stations_10_response.json()
    print(stations_10_json.keys())
else:
    print(f"Error: {stations_10_response.status_code}")

We can see that the JSON is made of two main elements, the `meta` and the `items`.
Meta contains metadata on things such as publisher and licenses.
In our case, we are interested in the items section, as this contains information on each station.
Let's look into the items in more detail.

In [ ]:
stations = stations_10_json['items']
stations[0]

We have successfully pulled in data from an API into our environment in JSON format.
After a while you get used to reading data in JSON format, but it isn't the nicest thing to view. Let's convert it to a DataFrame.
A DataFrame is a way of displaying data in a table format. It is much easier to look at and work with.

In [ ]:
stations_df = pd.DataFrame(stations)
stations_df

We have successfully pulled in data using an API. 
This isn't the most interesting dataset though. Let's access some slightly more interesting data and use a more complex query than just a limit.
We can define the `parameters` of our query and use them in our request. This is a nicer way of writing a request than making a longer and longer URL.

In [ ]:
readings_url = "https://environment.data.gov.uk/hydrology/data/readings.json"

params = {
    "station": "052d0819-2a32-47df-9b99-c243c9c8235b",
    "observedProperty": "waterFlow",
    "mineq-date": "2025-01-01",
    "maxeq-date": "2025-01-07"
}

reading_response = requests.get(readings_url, params=params)
print(reading_response.url)

if reading_response.status_code == 200:
    flow_json = reading_response.json()
    flow_df = pd.DataFrame(flow_json['items'])
    print("Success")
else:
    print("Error")

flow_df

To demonstrate what is possible, here is a few lines of code to visualise this data

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.lineplot(y="value", x="dateTime", data=flow_df)

plt.show()

Not a great looking plot. Let's neaten up the axes by adding some appropriate ticks and labels, as well as a title for the chart.

In [ ]:
sns.set_style("darkgrid")

g = sns.lineplot(y="value", x="dateTime", data=flow_df)

g.set_xticks([
    "2025-01-01T00:00:00",
    "2025-01-02T00:00:00",
    "2025-01-03T00:00:00",
    "2025-01-04T00:00:00",
    "2025-01-05T00:00:00",
    "2025-01-06T00:00:00",
    "2025-01-07T00:00:00",
    "2025-01-08T00:00:00"
],
[
    "1 Jan",
    "2 Jan",
    "3 Jan",
    "4 Jan",
    "5 Jan",
    "6 Jan",
    "7 Jan",
    "8 Jan"
])

plt.xlabel("Date")
plt.ylabel("Flow (m³/s)")
plt.title("Flow Time Series at Ulting Sarasota")

plt.show()